## Import

In [16]:
 %matplotlib inline

import random
import torch
from torch import nn, optim
import math
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.pyplot import axhline, axvline, grid, style
import seaborn
seaborn.set(style='ticks')
style.use('seaborn')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

px = 1/plt.rcParams['figure.dpi']  # inches to pixels

## Data

In [17]:
X = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1).to(device)
y = X.pow(2)*torch.sin(X*10) + 0.1 * torch.rand(X.size()).to(device)

## Model and Training Definition

In [18]:
random.seed(12345)
torch.manual_seed(12345)
N = 1000
C = 1 
I = 1 

learning_rate = 1e-3
lambda_l2 = 1e-5

model = nn.Sequential(
    nn.Linear(1, 300, bias = True),
    nn.ReLU(),
    nn.Linear(300, 1, bias = True)
)
model.to(device)

criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2)

# Training def
def epoch(epoch):

    model.to(device)
    y_pred = model(X)

    loss = criterion(y_pred, y)
    score, predicted = torch.max(y_pred, 1)    
    acc = (y == predicted).sum().float() / len(y)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    return loss, y_pred

## Training

In [21]:
%matplotlib tk

numbers = X[:,:].data.cpu().numpy()

fig, (ax, axloss) = plt.subplots(ncols=2, figsize=(960*px, 480*px))

ax.set(xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
ax.set_xlabel('x')
ax.set_ylabel('y')

axloss.set(xlim=(0, 1000), ylim=(0, 1))
axloss.set_xlabel('Epoch')
axloss.set_ylabel('Loss')

line1 = ax.plot([], [], color='tomato', lw=3)[0]
line2 = axloss.plot([], [], 'b-', lw=3)[0]

sc = ax.scatter(X.cpu().numpy()[:, 0], y.cpu().numpy()[:, 0], s=20, cmap=plt.cm.Paired_r)

titleLoss = axloss.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=axloss.transAxes, ha="center")

epochs = np.arange(0, 1000)
yloss = np.zeros(1000)

def animate(i):
    l, y_pred = epoch(i)          
    yloss[i] = l.item()  

    line1.set_data(numbers, y_pred.detach().cpu())
    line2.set_data(epochs[:i], yloss[:i])
    titleLoss.set_text("Epoch %d / Loss %.3f" % (i, l.item()))

    return [line1, line2, titleLoss]

anim = FuncAnimation(fig, animate, frames = epochs, interval=1, repeat=False, blit = True)
# anim.save('./fig/training_regression1.gif', writer='pillow', fps=30, dpi = 100)
plt.show()

## Extrapolation

In [22]:
%matplotlib tk

XTest = torch.unsqueeze(torch.linspace(-2, 2, steps=200), dim=1).to(device)
yTest = model(XTest)

fig, ax = plt.subplots(figsize=(480*px, 480*px))

ax.set(xlim=(-2.1, 2.1), ylim=(-2.1, 2.1))
ax.set_xlabel('x')
ax.set_ylabel('y')
line1 = ax.plot([], [], color='tomato', lw=3)[0]

sc = ax.scatter(X.cpu().numpy()[:, 0], y.cpu().numpy()[:, 0], s=20, cmap=plt.cm.Paired_r)

numbers = XTest[:,:].data.cpu().numpy()
line1 = ax.plot(numbers, yTest.detach().cpu(), color='tomato', lw=3)[0]

fig.show()